In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [3]:
# %run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_full_matrix_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', other_features=True)

data = Data('l')
data.generate_data('letters_word_accentuation_train',
                   'letters_word_accentuation_test',
                   'letters_word_accentuation_validate')

READING CONTENT...
CONTENT READ SUCCESSFULLY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
GENERATING X AND y...
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
GENERATION SUCCESSFUL!


In [7]:
pos = 0
print(len(feature_dictionary))
print(np.array(test2[0][0]).shape)
print(np.array(test2[0][1]).shape)
# for el in test2[0][0]:
# #     print(el)
#     print (data.decode_x(el, dictionary))
#     print(data.decode_x_other_features(feature_dictionary, [el]))

# for el in test2[1]:
#     print(el)

10
(16, 23, 36)
(16, 140)


In [6]:
# print (X_train.shape)
# print (X_test.shape)
# print (X_validate.shape)
pos = 0
# print (decode_input(X_train[pos], dictionary))
# print (decode_X_features(feature_dictionary, [X_other_features_train[pos]]))
# print(decode_position(y_train[pos]))
# print('------------------------------------')

print (data.x_train.shape)
print (data.x_test.shape)
print (data.x_validate.shape)
# pos = 2
print (data.decode_x(data.x_train[pos], dictionary))
print (data.decode_x_other_features(feature_dictionary, [data.x_other_features_train[pos]]))
print(data.x_train[pos])
print(data.decode_y(data.y_train[pos]))
print(dictionary)

(482209, 23, 36)
(52058, 23, 36)
(54222, 23, 36)


NameError: name 'dictionary' is not defined

In [4]:
data.x_train = np.concatenate((data.x_train, data.x_test), axis=0)
data.x_other_features_train = np.concatenate((data.x_other_features_train, data.x_other_features_test), axis=0)
data.y_train = np.concatenate((data.y_train, data.y_test), axis=0)

In [4]:
num_examples = len(data.x_train) # training set size
nn_output_dim = 10
nn_hdim = 516
batch_size = 16
actual_epoch = 20
num_fake_epoch = 20




conv_input_shape=(23, 36)
othr_input = (140, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(46, (3), padding='same', activation='relu')(x_conv)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=[actual_accuracy,])
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [5]:
history = model.fit_generator(data.generator('train', batch_size), 
                              data.x_train.shape[0]/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch, 
                              validation_data=data.generator('test', batch_size), 
                              validation_steps=data.x_test.shape[0]/(batch_size * num_fake_epoch))

Epoch 1/400
1345/1344 [==============================] - 18s - loss: 0.0482 - actual_accuracy: 0.6024 - val_loss: 0.0336 - val_actual_accuracy: 0.7508
Epoch 2/400
1345/1344 [==============================] - 24s - loss: 0.0335 - actual_accuracy: 0.7468 - val_loss: 0.0281 - val_actual_accuracy: 0.7914
Epoch 3/400
1345/1344 [==============================] - 25s - loss: 0.0281 - actual_accuracy: 0.7883 - val_loss: 0.0267 - val_actual_accuracy: 0.8087
Epoch 4/400
1345/1344 [==============================] - 25s - loss: 0.0251 - actual_accuracy: 0.8113 - val_loss: 0.0225 - val_actual_accuracy: 0.8386
Epoch 5/400
1345/1344 [==============================] - 28s - loss: 0.0222 - actual_accuracy: 0.8307 - val_loss: 0.0218 - val_actual_accuracy: 0.8397
Epoch 6/400
1345/1344 [==============================] - 35s - loss: 0.0207 - actual_accuracy: 0.8432 - val_loss: 0.0218 - val_actual_accuracy: 0.8409
Epoch 7/400
1345/1344 [==============================] - 43s - loss: 0.0187 - actual_accuracy:

1345/1344 [==============================] - 30s - loss: 0.0095 - actual_accuracy: 0.9312 - val_loss: 0.0169 - val_actual_accuracy: 0.8930
Epoch 55/400
1345/1344 [==============================] - 30s - loss: 0.0092 - actual_accuracy: 0.9320 - val_loss: 0.0161 - val_actual_accuracy: 0.8972
Epoch 56/400
1345/1344 [==============================] - 30s - loss: 0.0090 - actual_accuracy: 0.9349 - val_loss: 0.0187 - val_actual_accuracy: 0.8831
Epoch 57/400
1345/1344 [==============================] - 29s - loss: 0.0090 - actual_accuracy: 0.9347 - val_loss: 0.0167 - val_actual_accuracy: 0.8888
Epoch 58/400
1345/1344 [==============================] - 30s - loss: 0.0085 - actual_accuracy: 0.9378 - val_loss: 0.0194 - val_actual_accuracy: 0.8762
Epoch 59/400
1345/1344 [==============================] - 31s - loss: 0.0088 - actual_accuracy: 0.9351 - val_loss: 0.0204 - val_actual_accuracy: 0.8765
Epoch 60/400
1345/1344 [==============================] - 31s - loss: 0.0090 - actual_accuracy: 0.935

1345/1344 [==============================] - 33s - loss: 0.0096 - actual_accuracy: 0.9324 - val_loss: 0.0230 - val_actual_accuracy: 0.8620
Epoch 108/400
1345/1344 [==============================] - 33s - loss: 0.0087 - actual_accuracy: 0.9390 - val_loss: 0.0207 - val_actual_accuracy: 0.8769
Epoch 109/400
1345/1344 [==============================] - 33s - loss: 0.0091 - actual_accuracy: 0.9340 - val_loss: 0.0205 - val_actual_accuracy: 0.8758
Epoch 110/400
1345/1344 [==============================] - 33s - loss: 0.0094 - actual_accuracy: 0.9322 - val_loss: 0.0198 - val_actual_accuracy: 0.8785
Epoch 111/400
1345/1344 [==============================] - 33s - loss: 0.0091 - actual_accuracy: 0.9349 - val_loss: 0.0212 - val_actual_accuracy: 0.8639
Epoch 112/400
1345/1344 [==============================] - 32s - loss: 0.0096 - actual_accuracy: 0.9308 - val_loss: 0.0197 - val_actual_accuracy: 0.8842
Epoch 113/400
1345/1344 [==============================] - 32s - loss: 0.0100 - actual_accuracy:

1345/1344 [==============================] - 35s - loss: 0.0109 - actual_accuracy: 0.9234 - val_loss: 0.0196 - val_actual_accuracy: 0.8827
Epoch 161/400
1345/1344 [==============================] - 38s - loss: 0.0100 - actual_accuracy: 0.9267 - val_loss: 0.0185 - val_actual_accuracy: 0.8861
Epoch 162/400
1345/1344 [==============================] - 37s - loss: 0.0101 - actual_accuracy: 0.9285 - val_loss: 0.0203 - val_actual_accuracy: 0.8777
Epoch 163/400
1345/1344 [==============================] - 41s - loss: 0.0109 - actual_accuracy: 0.9239 - val_loss: 0.0204 - val_actual_accuracy: 0.8762
Epoch 164/400
1345/1344 [==============================] - 47s - loss: 0.0098 - actual_accuracy: 0.9304 - val_loss: 0.0244 - val_actual_accuracy: 0.8531
Epoch 165/400
1345/1344 [==============================] - 46s - loss: 0.0108 - actual_accuracy: 0.9243 - val_loss: 0.0220 - val_actual_accuracy: 0.8658
Epoch 166/400
1345/1344 [==============================] - 47s - loss: 0.0093 - actual_accuracy:

1345/1344 [==============================] - 35s - loss: 0.0105 - actual_accuracy: 0.9260 - val_loss: 0.0207 - val_actual_accuracy: 0.8692
Epoch 214/400
1345/1344 [==============================] - 35s - loss: 0.0118 - actual_accuracy: 0.9154 - val_loss: 0.0209 - val_actual_accuracy: 0.8742
Epoch 215/400
1345/1344 [==============================] - 36s - loss: 0.0117 - actual_accuracy: 0.9172 - val_loss: 0.0203 - val_actual_accuracy: 0.8758
Epoch 216/400
1345/1344 [==============================] - 36s - loss: 0.0128 - actual_accuracy: 0.9126 - val_loss: 0.0201 - val_actual_accuracy: 0.8727
Epoch 217/400
1345/1344 [==============================] - 35s - loss: 0.0114 - actual_accuracy: 0.9197 - val_loss: 0.0192 - val_actual_accuracy: 0.8777
Epoch 218/400
1345/1344 [==============================] - 36s - loss: 0.0115 - actual_accuracy: 0.9199 - val_loss: 0.0200 - val_actual_accuracy: 0.8796
Epoch 219/400
1345/1344 [==============================] - 36s - loss: 0.0108 - actual_accuracy:

1345/1344 [==============================] - 36s - loss: 0.0122 - actual_accuracy: 0.9178 - val_loss: 0.0225 - val_actual_accuracy: 0.8685
Epoch 267/400
1345/1344 [==============================] - 38s - loss: 0.0122 - actual_accuracy: 0.9175 - val_loss: 0.0220 - val_actual_accuracy: 0.8673
Epoch 268/400
1345/1344 [==============================] - 38s - loss: 0.0112 - actual_accuracy: 0.9217 - val_loss: 0.0241 - val_actual_accuracy: 0.8593
Epoch 269/400
1345/1344 [==============================] - 36s - loss: 0.0125 - actual_accuracy: 0.9138 - val_loss: 0.0233 - val_actual_accuracy: 0.8613
Epoch 270/400
1345/1344 [==============================] - 37s - loss: 0.0124 - actual_accuracy: 0.9162 - val_loss: 0.0263 - val_actual_accuracy: 0.8451
Epoch 271/400
1345/1344 [==============================] - 38s - loss: 0.0115 - actual_accuracy: 0.9218 - val_loss: 0.0219 - val_actual_accuracy: 0.8750
Epoch 272/400
1345/1344 [==============================] - 39s - loss: 0.0125 - actual_accuracy:

1345/1344 [==============================] - 51s - loss: 0.0133 - actual_accuracy: 0.9050 - val_loss: 0.0241 - val_actual_accuracy: 0.8505
Epoch 320/400
1345/1344 [==============================] - 55s - loss: 0.0140 - actual_accuracy: 0.9014 - val_loss: 0.0221 - val_actual_accuracy: 0.8685
Epoch 321/400
1345/1344 [==============================] - 58s - loss: 0.0134 - actual_accuracy: 0.9059 - val_loss: 0.0230 - val_actual_accuracy: 0.8597
Epoch 322/400
1345/1344 [==============================] - 54s - loss: 0.0141 - actual_accuracy: 0.9035 - val_loss: 0.0245 - val_actual_accuracy: 0.8543
Epoch 323/400
1345/1344 [==============================] - 52s - loss: 0.0140 - actual_accuracy: 0.9034 - val_loss: 0.0206 - val_actual_accuracy: 0.8762
Epoch 324/400
1345/1344 [==============================] - 50s - loss: 0.0123 - actual_accuracy: 0.9167 - val_loss: 0.0215 - val_actual_accuracy: 0.8738
Epoch 325/400
1345/1344 [==============================] - 47s - loss: 0.0131 - actual_accuracy:

1345/1344 [==============================] - 45s - loss: 0.0134 - actual_accuracy: 0.9062 - val_loss: 0.0215 - val_actual_accuracy: 0.8704
Epoch 373/400
1345/1344 [==============================] - 40s - loss: 0.0125 - actual_accuracy: 0.9152 - val_loss: 0.0197 - val_actual_accuracy: 0.8857
Epoch 374/400
1345/1344 [==============================] - 39s - loss: 0.0144 - actual_accuracy: 0.9039 - val_loss: 0.0234 - val_actual_accuracy: 0.8673
Epoch 375/400
1345/1344 [==============================] - 39s - loss: 0.0129 - actual_accuracy: 0.9146 - val_loss: 0.0198 - val_actual_accuracy: 0.8850
Epoch 376/400
1345/1344 [==============================] - 39s - loss: 0.0133 - actual_accuracy: 0.9109 - val_loss: 0.0207 - val_actual_accuracy: 0.8750
Epoch 377/400
1345/1344 [==============================] - 39s - loss: 0.0132 - actual_accuracy: 0.9119 - val_loss: 0.0222 - val_actual_accuracy: 0.8735
Epoch 378/400
1345/1344 [==============================] - 40s - loss: 0.0139 - actual_accuracy:

In [7]:
name = '20_test_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [4]:
#data = Data('l')
content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()

In [6]:
accented_vowels

['à', 'á', 'ä', 'é', 'ë', 'ì', 'í', 'î', 'ó', 'ô', 'ö', 'ú', 'ü']